In [1]:
from pipetools import pipe
from IPython.display import display

In [36]:
from model import *

In [37]:
# 实例化模型
sys = RCsys()
sys

In [38]:
# 参数替换字典
params = {
        sp.symbols('resistor_R'): 1.0,  # 电阻值
        sp.symbols('source_V'): 1.0,  # 电源电压
        sp.symbols('capacitor_C'): 1.0  # 电容值
}
params

{resistor_R: 1.0, source_V: 1.0, capacitor_C: 1.0}

In [39]:
equations = [eq.subs(params) for eq in sys.equations]
eqs_diff = [eq for eq in equations if eq.has(sp.Derivative)]
eqs_algebraic = [eq for eq in equations if not eq.has(sp.Derivative)]

eqs_algebraic > pipe | len | display
eqs_algebraic

21

[True,
 Eq(0, resistor_n_i + resistor_p_i),
 True,
 Eq(-resistor_n_v + resistor_p_v, 1.0*resistor_p_i),
 True,
 Eq(0, capacitor_n_i + capacitor_p_i),
 True,
 Eq(-capacitor_n_v + capacitor_p_v, V_C(t)),
 True,
 Eq(0, source_n_i + source_p_i),
 True,
 Eq(1.0, -source_n_v + source_p_v),
 Eq(ground_g_v, 0),
 Eq(source_p_v, resistor_p_v),
 Eq(resistor_p_i + source_p_i, 0),
 Eq(resistor_n_v, capacitor_p_v),
 Eq(capacitor_p_i + resistor_n_i, 0),
 Eq(capacitor_n_v, source_n_v),
 Eq(capacitor_n_i + source_n_i, 0),
 Eq(capacitor_n_v, ground_g_v),
 Eq(capacitor_n_i + ground_g_i, 0)]

In [40]:
# 收集所有代数变量，识别微分变量
vrs = set().union(*[eq.free_symbols for eq in equations]) 
# * 解包list[], 然后空集合与其并集,得到vrs集合，用集合去重 集合里相同元素会自动去重
vrs_diff = {eq.lhs.args[0] for eq in eqs_diff}
vrs_algebraic = list(vrs - vrs_diff - {t})

vrs_diff > pipe | display
vrs_algebraic > pipe | display

{V_C(t)}

[source_n_i,
 capacitor_p_v,
 ground_g_i,
 resistor_p_i,
 capacitor_n_i,
 ground_g_v,
 source_p_v,
 resistor_p_v,
 source_n_v,
 source_p_i,
 capacitor_p_i,
 resistor_n_v,
 capacitor_n_v,
 resistor_n_i]

**ask:** 请认真阅读@10_2_Rc.py ，vrs = set().union(*[eq.free_symbols for eq in equations]) 这条语句中的free_symbols是什么意思，得到列表[]后，[]前的*号是什么意思？union是什么意思？你能给我举一个非常简单的例子说明这条语句吗？<br>

**AI answer:**<br>
文字略,举例代码如下一个cell

In [41]:
import sympy as sp

# 定义符号
x, y, z = sp.symbols('x y z')

# 创建两个方程
eq1 = sp.Eq(x + 2*y, 0)
eq2 = sp.Eq(y - z, 5)

# 将方程放入列表
equations = [eq1, eq2]

# 我们要解释的语句
vrs = set().union(*[eq.free_symbols for eq in equations])

print(f"eq1.free_symbols: {eq1.free_symbols}")
print(f"eq2.free_symbols: {eq2.free_symbols}")
print(f"列表推导式的结果: {[eq.free_symbols for eq in equations]}")
print(f"最终 vrs 的结果: {vrs}")

eq1.free_symbols: {y, x}
eq2.free_symbols: {z, y}
列表推导式的结果: [{y, x}, {z, y}]
最终 vrs 的结果: {y, z, x}


In [42]:
{eqs_diff[0].lhs.args[0]} > pipe | display

{eqs_diff[0].lhs.args[0]} > pipe | type | display

eqs_diff[0].lhs.args[0] > pipe | type | display

{V_C(t)}

set

V_C

In [43]:
# 求解代数方程组
try:
    sol_algebraic = sp.solve(eqs_algebraic, vrs_algebraic, dict=True)[0]
except IndexError:  # 添加存在性检查
    print("代数方程求解失败，当前方程组：")
    for i, eq in enumerate(eqs_algebraic):
        print(f"Eq{i}: {eq}")
    raise RuntimeError("无法求解代数方程组，请检查设备元件连接关系")

In [44]:
sol_algebraic > pipe | len | display
sol_algebraic > pipe | display

14

{capacitor_n_i: V_C(t) - 1.0,
 capacitor_n_v: 0.0,
 capacitor_p_i: 1.0 - V_C(t),
 capacitor_p_v: V_C(t),
 ground_g_i: 1.0 - V_C(t),
 ground_g_v: 0.0,
 resistor_n_i: V_C(t) - 1.0,
 resistor_n_v: V_C(t),
 resistor_p_i: 1.0 - V_C(t),
 resistor_p_v: 1.00000000000000,
 source_n_i: 1.0 - V_C(t),
 source_n_v: 0.0,
 source_p_i: V_C(t) - 1.0,
 source_p_v: 1.00000000000000}

In [45]:
# 化简微分方程
eqs_ode = [eq.subs(sol_algebraic).doit() for eq in eqs_diff]
eqs_diff > pipe | display
eqs_ode > pipe | display


[Eq(Derivative(V_C(t), t), 1.0*capacitor_p_i)]

[Eq(Derivative(V_C(t), t), 1.0 - 1.0*V_C(t))]

In [46]:
# 提取ODE表达式
v_c = sp.Function('V_C')(t)
v_c > pipe | display
v_c > pipe | type | display

V_C(t)

V_C

In [47]:
ode_rhs = eqs_ode[0].rhs
ode_rhs > pipe | display
ode_rhs > pipe | type | display

1.0 - 1.0*V_C(t)

sympy.core.add.Add

In [48]:
# 创建数值计算函数
dydt = sp.lambdify((t, v_c), ode_rhs, modules='numpy')
dydt > pipe | display
dydt > pipe | type | display

<function _lambdifygenerated(t, _Dummy_43)>

function

In [49]:
# 求解ODE
t_span = (0, 10.0)        # 仿真时间段 0-10秒
u0 = [0.0]                # 微分变量电容电压初始值
np.linspace(*t_span, 100) # * 解包元组 等价于np.linspace(0, 10.0, 100) 


array([ 0.        ,  0.1010101 ,  0.2020202 ,  0.3030303 ,  0.4040404 ,
        0.50505051,  0.60606061,  0.70707071,  0.80808081,  0.90909091,
        1.01010101,  1.11111111,  1.21212121,  1.31313131,  1.41414141,
        1.51515152,  1.61616162,  1.71717172,  1.81818182,  1.91919192,
        2.02020202,  2.12121212,  2.22222222,  2.32323232,  2.42424242,
        2.52525253,  2.62626263,  2.72727273,  2.82828283,  2.92929293,
        3.03030303,  3.13131313,  3.23232323,  3.33333333,  3.43434343,
        3.53535354,  3.63636364,  3.73737374,  3.83838384,  3.93939394,
        4.04040404,  4.14141414,  4.24242424,  4.34343434,  4.44444444,
        4.54545455,  4.64646465,  4.74747475,  4.84848485,  4.94949495,
        5.05050505,  5.15151515,  5.25252525,  5.35353535,  5.45454545,
        5.55555556,  5.65656566,  5.75757576,  5.85858586,  5.95959596,
        6.06060606,  6.16161616,  6.26262626,  6.36363636,  6.46464646,
        6.56565657,  6.66666667,  6.76767677,  6.86868687,  6.96

In [50]:
sol = solve_ivp(dydt, t_span, u0, t_eval=np.linspace(*t_span, 100))
sol > pipe | type | display
sol

scipy.integrate._ivp.ivp.OdeResult

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.010e-01 ...  9.899e+00  1.000e+01]
        y: [[ 0.000e+00  9.608e-02 ...  9.999e-01  9.999e-01]]
      sol: None
 t_events: None
 y_events: None
     nfev: 74
     njev: 0
      nlu: 0

In [51]:
# 保存仿真结果数据
save_outs(sol, 0, "time\\s", "Vc\\V")

数据已成功保存至 sim_outs.xlsx


In [52]:
# 可视化结果
plot(sol.t, sol.y[0], label="Capacitor Voltage Vc",
     x_label="time / s", y_label="Vc / V")

# print("---- End ----")

KeyboardInterrupt: 

In [53]:
sol[sys.resistor_n_i]

AttributeError: 'RCsys' object has no attribute 'resistor_n_i'